In [13]:
from src.Molecule import R_group, Molecule, FinalMolecule
from flask import Flask, jsonify, request

In [14]:
global assayed_molecules
assayed_molecules = {}
global money
money = [100000.0]

global time
time = [30.0]

In [15]:
def tuple2str(tuple_in):
    """Converts a tuple into a string

    :param tuple_in: tuple to convery
    :type tuple_in: tuple
    :returns: concatenated string version of the tuple
    :rtype: str
    """
    string = ''
    for i in tuple_in:
        string += str(i)
    return string

In [16]:
def run_assays(r_group_1_id, r_group_2_id, assay_list):
    """Runs assays selected for a specific molecule, tracking the reduction of
    time and money as a result.
    The longest time of the assays being run is taken from the total amount of
    time.
    The sum of the cost of the assays is taken from the total amount of money.
    Pass R group IDs as queries: /assays?r1=A01&r2=B10... and Yes or No for
    each assay,
    ...&pic50=Yes&clearance_mouse=No&clearance_human=Yes&logd=No&pampa=Yes.

    :returns: A json dictionary of which molecules have been assayed, indexed
    by the concatenated string of their R Group IDs, with the values being
    which assays have been run and their values for all assays
    :rtype: json dict
    """

    # The prices of each assay
    assay_prices = {
        "pic50": 70.0,
        "clearance_mouse": 7000.0,
        "clearance_human": 9000.0,
        "logd": 1000.0,
        "pampa": 700.0,
    }
    # How long each assay takes in weeks
    assay_times = {
        "pic50": 1.0,
        "clearance_mouse": 3.0,
        "clearance_human": 3.5,
        "logd": 1.5,
        "pampa": 1.0,
    }
    # r_group_1_id = request.args.get('r1')
    # r_group_2_id = request.args.get('r2')
    # assay_list = []
    # Make a list of the assays being run
    # for label in ['pic50',
    #               'clearance_mouse',
    #               'clearance_human',
    #               'logd',
    #               'pampa']:
    #     if request.args.get(label) == "Yes":
    #         assay_list.append(label)


    drug_mol = FinalMolecule(r_group_1_id, r_group_2_id)
    drug_properties = {label: drug_mol.drug_properties()[label] for label in assay_list}
    molecule_key = tuple2str((r_group_1_id, r_group_2_id))
    if molecule_key not in assayed_molecules.keys():
        assayed_molecules[molecule_key] = drug_properties
    else:
        for label in assay_list:
            assayed_molecules[molecule_key][label] = drug_properties[label]
    if money[-1] - sum([assay_prices[p] for p in assay_list]) < 0:
        pass
    else:
        money.append(money[-1] - sum([assay_prices[p] for p in assay_list]))
    if time[-1] - max([assay_times[p] for p in assay_list]) < 0:
        pass
    else:
        time.append(time[-1] - max([assay_times[p] for p in assay_list]))
    return jsonify({"assay_backend_data": assayed_molecules})

In [17]:
result = run_assays("A01", "B01", ["clearance_human", "pampa"])

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [15]:
import pandas as pd  
csv_file = pd.read_csv('src/r_group_decomp.csv')
rgroup1 = "A01"
rgroup2 = "B01"
# def drug_properties(csv_file, rgroup1, rgroup2):
"""Selects properties of the final drug from the data."""
drug_properties = [
                    'pic50',
                    'clearance_mouse',
                    'clearance_human',
                    'logd',
                    'pampa'
                    ]
drug_property_dict = {}
for d in drug_properties:
    value = csv_file[csv_file['atag'] == rgroup1]
    value_two = value[csv_file['btag'] == rgroup2][d].iloc[0]
    drug_property_dict[d] = value_two
print(drug_property_dict)

{'pic50': '6.5', 'clearance_mouse': 'medium (5.6-30.5)', 'clearance_human': 'low (< 12)', 'logd': '0.3', 'pampa': 'low'}


<ipython-input-15-11184c5fc735>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  value_two = value[csv_file['btag'] == rgroup2][d].iloc[0]
